In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTEN
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, ClusterCentroids
from lazypredict.Supervised import LazyClassifier
from sklearn.utils import all_estimators 
from sklearn.base import ClassifierMixin
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
import time

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
import copy
from collections import Counter

cols_cible_type = ["TurnoutTimeSeconds", "TravelTimeSeconds", "PumpSecondsOnSite"]
cols_Data = [
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "PropertyType", "StopCode"],
]
# utilise -1 sinon crée bizarrement des Nan pour les valeurs à 0 aua lieu de mettre 0
cols_cible_bins = [
    {
        "bins": np.array([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20000]) * 60,
        "labels": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30],
    },
    {
        "bins": np.array([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20000]) * 60,
        "labels": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30],
    },
    {
        "bins": np.array([-1, 5, 10, 15, 30, 45, 60, 120, 180, 360, 1000000]) * 60,
        "labels": [5, 10, 15, 30, 45, 60, 120, 180, 360, 1000],
    },
]

cols_cible_bins_numbers = [4, 10, 9]

cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    # NumPumpsAttending??
]

# copie profonde, sinon la simple copie fait une copie des références des ss tableaux, et leur modif modifie l'original
cols_cible_minutes = copy.deepcopy(cols_cible)


def load_df(col_cible_type):
    df = pd.read_csv(
        f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False
    )
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df


def create_cols_cible_minutes(df, index, bins=-1):
    # Crée 3 nouvelles target pour chque min/mean/max, par tranche en minutes, pour la cible type actuelle
    for index_cible, col_cible in enumerate(cols_cible[index]):
        # bins = cols_cible_bins[index]["bins"]
        # print(bins)
        # labels = cols_cible_bins[index]["labels"]
        # print(labels)
        new_name = col_cible.replace("Seconds", "Minutes")
        print(new_name, col_cible)
        # df[new_name] = pd.cut(x=df[col_cible], bins=bins, labels=labels)
        num_bins = cols_cible_bins_numbers[index] if bins == -1 else bins
        new_bins = pd.qcut(df[col_cible], q=num_bins, precision=0)
        # 2. Afficher la répartition des observations dans chaque bin
        # 2. Modifier les labels pour qu'ils correspondent à la valeur haute de chaque intervalle
        df[new_name] = new_bins.apply(lambda x: x.right)  # Prendre la valeur haute de l'intervalle
        # 3. Afficher la répartition des observations dans chaque bin
        bin_counts = df[new_name].value_counts().sort_index()
        print(bin_counts)
        cols_cible_minutes[index][index_cible] = new_name
        print("cols_cible", cols_cible)
        print("cols_cible_minutes", cols_cible_minutes)
        
    display(df.head(3))
    return df

def Create_X(df_limited, index):
    print(df_limited.shape)
    # Crée X sans les target
    cols_to_remove = (
        [item for sublist in cols_cible for item in sublist]
        + cols_cible_minutes[index]
        + ["NumPumpsAttending"]
    )
    print("cols_to_remove", cols_to_remove)
    X = df_limited.drop(cols_to_remove, axis=1)
    # Ne conserve que certaines colonnes explicatives
    # Crée une liste des colonnes à conserver
    cols_to_keep = [
        col
        for col in X.columns
        if any(substring in col for substring in cols_Data[index])
    ]
    print("cols_to_keep", cols_to_keep)
    # Conserver uniquement ces colonnes
    X = X[cols_to_keep]

    display(X.head(2))
    print(X.shape)
    return X

def under_sampling(X_train, y_train):
    # Réchantillonne
    print("Resampling")
    print(len(X_train))
    start_time = time.time()
    print(y_train.value_counts())
    # sampling = RandomUnderSampler(sampling_strategy="auto", random_state=42)
    print("Smote")
    sampling = SMOTE(random_state=42)
    X_train_samp, y_train_samp = sampling.fit_resample(X_train, y_train)
    print(y_train_samp.value_counts())

    # print("EditedNearestNeighbours")
    # sampling = EditedNearestNeighbours(n_jobs=-1)
    # X_train_samp, y_train_samp = sampling.fit_resample(X_train_samp, y_train_samp)
    
    # print("ClusterCentroids")
    # sampling = ClusterCentroids(random_state=42)
    # X_train_samp, y_train_samp = sampling.fit_resample(X_train, y_train) # 35 minutes sur 3 cibles


    # print("RandomUnderSampler à 60%")
    # # Compte la taille initiale des classes avant SMOTE
    # class_counts_before_smote = Counter(y_train)
    # max_class_count_before_smote = max(class_counts_before_smote.values())
    # # Calcule 60% de la taille de la classe majoritaire initiale pour chaque classe pour appliquer le RandomUnderSampler
    # sampling_strategy = {label: int(0.60 * max_class_count_before_smote) for label in class_counts_before_smote}
    # # Appliquer RandomUnderSampler pour ajuster les classes 
    # undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
    # X_train_samp, y_train_samp = undersampler.fit_resample(X_train_samp, y_train_samp)


    end_time = time.time()
    print(y_train_samp.value_counts())
    print(f"Durée de RandomUnderSampler : {end_time - start_time:.2f} secondes")
    print(len(X_train_samp))
    return X_train_samp, y_train_samp

In [47]:
# vérification de l'équilibrage des classes par cible

cols_cible_filter = [
    "TurnoutTimeSeconds_mean",
    "TravelTimeSeconds_mean",
    "PumpSecondsOnSite_mean",
]

# for index, name in enumerate(cols_cible_type):
#     df = load_df(name)
#     df = create_cols_cible_minutes(df, index, name)

#     for index_cible, col_cible in enumerate(cols_cible[index]):
#         if not (col_cible in cols_cible_filter):
#             continue
#         print("---------------------------------------------------")
#         minute_col = cols_cible_minutes[index][index_cible]
#         print(col_cible, ">>", minute_col)
#         # Vérification de l'équilibre des classes pour Target
#         print("Distribution des classes pour Target :")
#         print(df[minute_col].value_counts())

#         print("duplicated", df.duplicated().sum())

print("///////////////////////////////////////////////////////////////")

for index, name in enumerate(cols_cible_type):
    df = load_df(name)

    for index_cible, col_cible in enumerate(cols_cible[index]):
        # if not (col_cible in cols_cible_filter):
        #     continue
        print("---------------------------------------------------")
        # Vérification de l'équilibre des classes pour Target
        print("Distribution qcut pour Target :", col_cible)

        # num_bins = 5  # Nombre de bins (quantiles)
        for num_bins in range(2, 12):  # Nombre de bins (quantiles)
            y_train_bins = pd.qcut(df[col_cible], q=num_bins, precision=0)
            # 2. Afficher la répartition des observations dans chaque bin
            # 2. Modifier les labels pour qu'ils correspondent à la valeur haute de chaque intervalle
            y_train_labels = y_train_bins.apply(lambda x: x.right)  # Prendre la valeur haute de l'intervalle
            # 3. Afficher la répartition des observations dans chaque bin
            bin_counts = y_train_labels.value_counts().sort_index()
            print(bin_counts)

        # y_train_bins = pd.cut(df[col_cible], bins=num_bins, precision=0)
        # # 2. Afficher la répartition des observations dans chaque bin
        # bin_counts = y_train_bins.value_counts().sort_index()
        # print(bin_counts)

        print("duplicated", df.duplicated().sum())

///////////////////////////////////////////////////////////////
---------------------------------------------------
Distribution qcut pour Target : TurnoutTimeSeconds_min
TurnoutTimeSeconds_min
66.00      800263
1192.00    791264
Name: count, dtype: int64
TurnoutTimeSeconds_min
54.00      532265
79.00      537073
1192.00    522189
Name: count, dtype: int64
TurnoutTimeSeconds_min
48.00      414285
66.00      385978
87.00      396402
1192.00    394862
Name: count, dtype: int64
TurnoutTimeSeconds_min
43.00      327710
59.00      313535
74.00      332824
94.00      309889
1192.00    307569
Name: count, dtype: int64
TurnoutTimeSeconds_min
39.00      266015
54.00      266250
66.00      267998
79.00      269075
98.00      257215
1192.00    264974
Name: count, dtype: int64
TurnoutTimeSeconds_min
37.00      238411
51.00      232624
61.00      215107
71.00      225540
84.00      240751
103.00     219175
1192.00    219919
Name: count, dtype: int64
TurnoutTimeSeconds_min
34.00      200793
48.00   

In [4]:
cols_cible_filter = [
    "TurnoutTimeSeconds_mean",
    "TravelTimeSeconds_mean",
    "PumpSecondsOnSite_mean",
]

ignore_classifiers = [
    "CalibratedClassifierCV",
    "CalibratedClassifierCV",
    "CategoricalNB",
    "ClassifierChain",
    "ComplementNB",
    "FixedThresholdClassifier",
    "GaussianProcessClassifier",
    "GradientBoostingClassifier",
    "HistGradientBoostingClassifier",
    "LabelPropagation",
    "LabelSpreading",
    "LinearSVC",
    "LogisticRegressionCV",
    "NuSVC",
    "SVC",
]

CLASSIFIERS = [
    estimator
    for estimator in all_estimators()
    if (
        issubclass(estimator[1], ClassifierMixin)
        and (not (estimator[0] in ignore_classifiers))
    )
]

all_results = pd.DataFrame()

for index, name in enumerate(cols_cible_type):
    df = load_df(name)

    # df = df[df.CalYear > 6]
    # pd.DataFrame(pd.cut(X['Age'], bins = [0, 12, 18, 30, 50, 65, np.max(df.Age)], labels=['Kid','Adolescent','Adult-','Adult','Adult+','Senior']))
    # df['age_cat'] = pd.cut(x = df['age'], bins = [19,24,28,32,36,40], labels = ['19-24', '25-28', '29-32', '33-36', '37-40'],
    #                    include_lowest = True)
    # X = df[["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
    #               "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"]]
    # plt.boxplot([df.TurnoutTimeSeconds_min, df.TurnoutTimeSeconds_mean, df.TurnoutTimeSeconds_max])
    # plt.show()
    # plt.boxplot([df.TravelTimeSeconds_min, df.TravelTimeSeconds_mean, df.TravelTimeSeconds_max])
    # plt.show()
    # plt.figure(figsize=(5, 10))
    # plt.boxplot([df.PumpSecondsOnSite_min, df.PumpSecondsOnSite_mean, df.PumpSecondsOnSite_max])
    # plt.show()
    # Auto ML
    for year_floor in range(13, 16):        
        df_limited = df[df.CalYear >= year_floor]  # >= 2021
        # test avec différents bins
        for bins in [4, 5, 8, 10]:
            df_limited = create_cols_cible_minutes(df_limited, index, bins)
            print("-----------------------------------------------------")
            print("year", 2024 - 16 + year_floor, "bins", bins)
            print("-----------------------------------------------------")
            X = Create_X(df_limited, index)
            for index_cible, col_cible in enumerate(cols_cible[index]):
                if not (col_cible in cols_cible_filter):
                    continue
                minute_col = cols_cible_minutes[index][index_cible]
                print("/////", col_cible, ">>", minute_col)
                y = df_limited[minute_col]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                # # Réchantillonne
                # print("TomekLinks")
                # print(len(X_train))
                # start_time = time.time()
                # tomek = TomekLinks(n_jobs=-1)
                # X_train_smote, y_train_smote = tomek.fit_resample(X_train, y_train)
                # end_time = time.time()
                # print(f"Durée de TomekLinks : {end_time - start_time:.2f} secondes")
                # print(len(X_train_smote))
                # # Réchantillonne
                # print("SMOTETomek")
                # print(len(X_train))
                # start_time = time.time()
                # smote = SMOTETomek(n_jobs=-1)
                # X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
                # end_time = time.time()
                # print(f"Durée de SMOTETomek : {end_time - start_time:.2f} secondes")
                # print(len(X_train_smote))
                # Réchantillonne
                # X_train_samp, y_train_samp = under_sampling(X_train, y_train)
                X_train_samp, y_train_samp = X_train, y_train
                # LazyPredict
                myCLASSIFIERS = CLASSIFIERS
                print(myCLASSIFIERS)  
                # myCLASSIFIERS = myCLASSIFIERS[4:6]
                reg = LazyClassifier(
                    verbose=2,
                    ignore_warnings=False,
                    custom_metric=None,
                    classifiers=myCLASSIFIERS,
                )       
                models, predictions = reg.fit(X_train_samp, X_test, y_train_samp, y_test)
                # Ajoute le nom de la colonne
                models["Target"] = minute_col
                models["Bins"] = bins
                models["Year floor"] = 2024 - 16 + year_floor
                # Concaténe dans les résultats
                all_results = pd.concat([all_results, models], axis=0)
                display(models)
                models.to_csv(
                    f"../data/_autoML_classifier_{minute_col} {bins} {year_floor}.csv", sep=";", index=True
                )
                # break
            # break
all_results = all_results.sort_values(by=["Target", "F1 Score"], ascending=False)
all_results.to_csv(f"../data/_autoML_classifier.csv", sep=";", index=True)

# tester automl avant et après PCA et aussi gridsearch pour comparer
# save du PCA

TurnoutTimeMinutes_min TurnoutTimeSeconds_min
TurnoutTimeMinutes_min
44.00      112688
64.00      114758
83.00      106502
1192.00    110299
Name: count, dtype: int64
cols_cible [['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max']]
cols_cible_minutes [['TurnoutTimeMinutes_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max']]
TurnoutTimeMinutes_mean TurnoutTimeSeconds_mean
TurnoutTimeMinutes_mean
53.00      114291
70.00      113074
89.00      108903
1192.00    107979
Name: count, dtype: int64
cols_cible [['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeS

,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,PropertyType_6,PropertyType_7,PropertyType_8,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,NumPumpsAttending,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max,TurnoutTimeMinutes_min,TurnoutTimeMinutes_mean,TurnoutTimeMinutes_max
1147280,13,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1.00,0,0,0,1,1,0,0,0,1,0,1,0,2880.00,2880.00,2880.00,48.00,48.00,48.00,133.00,133.00,133.00,64.00,53.00,57.00
1147281,13,0,0,0,0,1,0,0,0,1,0,1,1,1,1,0,0,0,0,1,1.00,0,0,0,1,0,0,0,0,1,0,1,0,420.00,420.00,420.00,114.00,114.00,114.00,136.00,136.00,136.00,1192.00,1192.00,1192.00
1147282,13,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1.00,0,0,0,1,1,0,0,0,1,0,1,0,600.00,600.00,600.00,131.00,131.00,131.00,245.00,245.00,245.00,1192.00,1192.00,1192.00


-----------------------------------------------------
year 2021 bins 4
-----------------------------------------------------
(444247, 45)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'TurnoutTimeMinutes_min', 'TurnoutTimeMinutes_mean', 'TurnoutTimeMinutes_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
1147280,13,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
1147281,13,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0


(444247, 18)
///// TurnoutTimeSeconds_mean >> TurnoutTimeMinutes_mean
[('AdaBoostClassifier', <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>), ('BaggingClassifier', <class 'sklearn.ensemble._bagging.BaggingClassifier'>), ('BernoulliNB', <class 'sklearn.naive_bayes.BernoulliNB'>), ('DecisionTreeClassifier', <class 'sklearn.tree._classes.DecisionTreeClassifier'>), ('DummyClassifier', <class 'sklearn.dummy.DummyClassifier'>), ('ExtraTreeClassifier', <class 'sklearn.tree._classes.ExtraTreeClassifier'>), ('ExtraTreesClassifier', <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>), ('GaussianNB', <class 'sklearn.naive_bayes.GaussianNB'>), ('KNeighborsClassifier', <class 'sklearn.neighbors._classification.KNeighborsClassifier'>), ('LinearDiscriminantAnalysis', <class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>), ('LogisticRegression', <class 'sklearn.linear_model._logistic.LogisticRegression'>), ('MLPClassifier', <class 'sklearn.neural_network._multilayer_

...

 83%|████████▎ | 25/30 [08:16<02:21, 28.39s/it]

ROC AUC couldn't be calculated for RidgeClassifierCV
multi_class must be in ('ovo', 'ovr')
{'Model': 'RidgeClassifierCV', 'Accuracy': 0.1935075323290228, 'Balanced Accuracy': np.float64(0.17220776265137733), 'ROC AUC': None, 'F1 Score': 0.10213880580132043, 'Time taken': 0.452592134475708}


100%|██████████| 30/30 [08:19<00:00, 16.66s/it]

ROC AUC couldn't be calculated for SGDClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'SGDClassifier', 'Accuracy': 0.14789139225880993, 'Balanced Accuracy': np.float64(0.1367052363024588), 'ROC AUC': None, 'F1 Score': 0.10392525879251449, 'Time taken': 2.892306089401245}
SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'
VotingClassifier model failed to execute
VotingClassifier.__init__() missing 1 required positional argument: 'estimators'


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Target,Bins,Year floor
Model,,,,,,,,
MLPClassifier,0.21,0.19,None,0.12,42.66,PumpMinutesOnSite_mean,10,2023
RandomForestClassifier,0.21,0.18,None,0.12,4.24,PumpMinutesOnSite_mean,10,2023
ExtraTreesClassifier,0.21,0.18,None,0.12,4.30,PumpMinutesOnSite_mean,10,2023
DecisionTreeClassifier,0.21,0.18,None,0.12,0.24,PumpMinutesOnSite_mean,10,2023
ExtraTreeClassifier,0.21,0.18,None,0.12,0.16,PumpMinutesOnSite_mean,10,2023
BaggingClassifier,0.21,0.18,None,0.12,1.06,PumpMinutesOnSite_mean,10,2023
LinearDiscriminantAnalysis,0.19,0.17,None,0.12,0.31,PumpMinutesOnSite_mean,10,2023
LogisticRegression,0.19,0.17,None,0.11,1.09,PumpMinutesOnSite_mean,10,2023
RidgeClassifier,0.19,0.17,None,0.10,0.18,PumpMinutesOnSite_mean,10,2023


In [37]:
# Classifiers retenus pour optimisation, prend le top 5 de chaque cible et les entraine tous
# pour voir si au final on peut limiter à certains communs

# Pour TurnoutTimeSeconds
# KNeighborsClassifier
# DecisionTreeClassifier
# ExtraTreeClassifier
# ExtraTreesClassifier
# BaggingClassifier

# Pour TravelTimeSeconds
# MLPClassifier
# RandomForestClassifier
# BaggingClassifier
# ExtraTreesClassifier
# KNeighborsClassifier

# Pour PumpSecondsOnSite
# LinearDiscriminantAnalysis
# BaggingClassifier
# RandomForestClassifier
# ExtraTreeClassifier
# ExtraTreesClassifier

# On va effectuer un GridSearchCV pour :
# KNeighborsClassifier
# DecisionTreeClassifier
# ExtraTreeClassifier
# ExtraTreesClassifier
# BaggingClassifier
# MLPClassifier
# RandomForestClassifier
# LinearDiscriminantAnalysis

import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (
    ExtraTreesClassifier,
    BaggingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
)
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import (
    PassiveAggressiveClassifier,
    Perceptron,
    LogisticRegression,
    RidgeClassifier,
    RidgeClassifierCV,
    SGDClassifier,
)
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import NearestCentroid
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_validate

from sklearn.metrics import classification_report, accuracy_score, f1_score

# Hyperparamètres à tester pour chaque modèle
param_grid = {
    "KNeighborsClassifier": {
        "n_neighbors": [3, 5, 10, 15],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"],
        "p": [1, 2],
    },
    "DecisionTreeClassifier": {
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "criterion": ["gini", "entropy", "log_loss"],
    },
    "ExtraTreeClassifier": {
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "criterion": ["gini", "entropy", "log_loss"],
    },
    "ExtraTreesClassifier": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "criterion": ["gini", "entropy", "log_loss"],
    },
    "BaggingClassifier": {
        "n_estimators": [10, 50, 100],
        "max_samples": [0.5, 0.75, 1.0],
        "bootstrap": [True, False],
    },
    "RandomForestClassifier": {
        "n_estimators": [10, 50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "criterion": ["gini", "entropy"],
    },
    "RadiusNeighborsClassifier": {
        "radius": [0.5, 1.0, 2.0],
        "weights": ["uniform", "distance"],
    },
    "MLPClassifier": {
        "hidden_layer_sizes": [(50,), (100,), (50, 50), (100, 100)],
        "activation": ["relu", "tanh", "logistic"],
        "solver": ["adam", "sgd"],
        "learning_rate_init": [0.001, 0.01, 0.1],
        "learning_rate": ["constant", "adaptive"],
    },
    "PassiveAggressiveClassifier": {
        "C": [0.1, 1.0, 10.0],
        "max_iter": [1000, 2000],
    },
    "Perceptron": {
        "penalty": [None, "l2", "l1", "elasticnet"],
        "alpha": [0.0001, 0.001, 0.01],
    },
    "AdaBoostClassifier": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.1, 0.5, 1.0],
    },
    "LogisticRegression": {
        "penalty": ["l1", "l2", "elasticnet", "none"],
        "C": [0.1, 1.0, 10.0],
        "solver": ["liblinear", "saga"],
    },
    "LinearDiscriminantAnalysis": {
        "solver": ["svd", "lsqr", "eigen"],
        "shrinkage": [None, "auto"],
    },
    "RidgeClassifier": {
        "alpha": [0.1, 1.0, 10.0, 100.0],
    },
    "RidgeClassifierCV": {},
    "BernoulliNB": {
        "alpha": [0.01, 0.1, 1.0],
    },
    "SGDClassifier": {
        "alpha": [0.0001, 0.001, 0.01],
        "penalty": ["l2", "l1", "elasticnet"],
        "max_iter": [1000, 2000],
        'loss': ['hinge', 'log'],
    },
    "QuadraticDiscriminantAnalysis": {
        "reg_param": [0.0, 0.1, 0.5],
    },
    "NearestCentroid": {},
    "GaussianNB": {},
}

classifiers = {
    # "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "ExtraTreeClassifier": ExtraTreeClassifier(),
    # "ExtraTreesClassifier": ExtraTreesClassifier(),
    # "BaggingClassifier": BaggingClassifier(),
    # "RandomForestClassifier": RandomForestClassifier(),
    # "RadiusNeighborsClassifier": RadiusNeighborsClassifier(),
    # "MLPClassifier": MLPClassifier(),
    # "PassiveAggressiveClassifier": PassiveAggressiveClassifier(),
    # "Perceptron": Perceptron(),
    # "AdaBoostClassifier": AdaBoostClassifier(),
    # "LogisticRegression": LogisticRegression(),
    # "LinearDiscriminantAnalysis": LinearDiscriminantAnalysis(),
    # "RidgeClassifier": RidgeClassifier(),
    # "RidgeClassifierCV": RidgeClassifierCV(),
    # "BernoulliNB": BernoulliNB(),
    # "SGDClassifier": SGDClassifier(),
    # "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    # "NearestCentroid": NearestCentroid(),
    # "GaussianNB": GaussianNB(),
}

results = []
crossval = KFold(n_splits=5, random_state=42, shuffle=True)

for index, name in enumerate(cols_cible_type):
    df = load_df(name)

    for year_floor in range(13, 17):
        print("-----------------------------------------------------")
        print("year", 2024 - 16 + year_floor)
        print("-----------------------------------------------------")
        df_limited = df[df.CalYear >= year_floor]  # >= 2021
        df_limited = create_cols_cible_minutes(df_limited, index, 4)
        X = Create_X(df_limited, index)
        for index_cible, col_cible in enumerate(cols_cible[index]):
            minute_col = cols_cible_minutes[index][index_cible]
            print(
                "--------------------------------------------------------------------------------"
            )
            print("///////////////////////////", col_cible, ">>", minute_col)
            y = df_limited[minute_col]            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            # Réchantillonne
            # X_train_samp, y_train_samp = under_sampling(X_train, y_train)
            X_train_samp, y_train_samp = X_train, y_train

            # continue

            # Exécuter GridSearchCV pour chaque modèle
            for model_name, model in classifiers.items():
                # Controle run, car potentiellement long
                print("--------------------------------")
                print(f"GridSearchCV pour {model_name}")
                grid = GridSearchCV(
                    model,
                    param_grid[model_name],
                    scoring="f1_macro",
                    cv=crossval,
                    n_jobs=-1,
                    verbose=2,
                )
                # Fit et Predict
                grid.fit(X_train_samp, y_train_samp)

                test 
                y_pred = grid.best_estimator_.predict(X_test)

                y_pred = grid.predict(X_test)
                
                # Meilleur score et meilleurs hyperparamètres
                best_params = grid.best_params_
                best_score = grid.best_score_
                # affiche de suite les résultat
                print(f"Meilleurs paramètres {best_params}")
                print(f"Meilleur score {best_score}")
                # Accuracy
                accuracy = accuracy_score(y_test, y_pred)
                print(f"Accuracy : {accuracy}")
                # train et test score
                score_train = grid.score(X_train_samp, y_train_samp)
                print(f"Score train : {score_train}")
                score_test = grid.score(X_test, y_test)
                print(f"Score test : {score_test}")
                # Rapport détaillé
                detailled_report = classification_report(y_test, y_pred)
                print(f"Classification report :\n {detailled_report}")
                # Matrice de confusion
                ct = pd.crosstab(
                    y_test, y_pred, rownames=["Classe réelle"], colnames=["Classe prédite"]
                )
                display(ct)
                # Score de validation croisée
                #             scores = cross_validate(grid, X_train, y_train, cv=crossval, scoring=['accuracy', 'precision', 'recall', 'f1'])
                #             cv_test_accuracy = f" cv_test_accuracy : {scores['test_accuracy'].mean()} (+/-{scores['test_accuracy'].std()})"
                #             cv_test_precision
                #             cv_test_recall
                #             cv_test_f1

                # {'fit_time': array([1.7602632 , 1.3521595 , 1.53327417, 1.6628437 , 1.52980328]),
                #  'score_time': array([0.10070086, 0.10009599, 0.11566949, 0.09443331, 0.09455991]),
                #  'test_accuracy': array([0.87765306, 0.87768222, 0.8774344 , 0.87757839, 0.87759297]),
                #  'test_precision': array([0.36      , 0.43243243, 0.2745098 , 0.29032258, 0.38888889]),
                #  'test_recall': array([0.00107322, 0.00190794, 0.00166965, 0.00107335, 0.00250447]),
                #  'test_f1': array([0.00214005, 0.00379912, 0.00331911, 0.00213878, 0.00497689])}

                # print("[StackingClassifier]: \n Accuracy: %0.2f (+/- %0.2f)\n" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
                #       "F1 score: %0.2f (+/- %0.2f)" % (scores['test_f1'].mean(), scores['test_f1'].std()))
                # >>>
                # [StackingClassifier]:
                #  Accuracy: 0.76 (+/- 0.02)
                #  F1 score: 0.62 (+/- 0.04)

                # Ajouter au tableau des résultats
                results.append(
                    {
                        "Target": minute_col,
                        "Year floor": 2024 - 16 + year_floor,
                        "Model": model_name,
                        "Best Params": best_params,
                        "Best Score": best_score,
                        "Accuracy Test": accuracy,
                        "Score Train": score_train,
                        "Score Test": score_test,
                        "Detailled_report": detailled_report,
                        "Prédiction": ct,
                    }
                )
print(results)
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by=["Target", "Score Test"], ascending=False)
df_results.to_csv(f"../data/_GridSearch_classifier.csv", sep=";", index=True)

# p = plt.figure(figsize=(18,12))
# p = sns.set_theme(style="white")
# p = models=models.sort_values(by='Model Accuracy with GridSearch',ascending=False)
# p = sns.barplot(y= 'Model', x= 'Model Accuracy with GridSearch', data= models)
# for container in p.containers:
#     p.bar_label(container,label_type = 'edge',padding = 8,size = 20,color = "black",rotation = 0,
#     bbox={"boxstyle": "round", "pad": 0.6, "facecolor": "pink", "edgecolor": "Blue", "alpha": 1})
# plt.title('COMPARE THE MODEL')
# plt.xlabel('MODEL')
# plt.ylabel('Model Accuracy')
# plt.show()

[    -60      60     120     180     240     300     360     420     480
     540     600     900 1200000]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30]
TurnoutTimeMinutes_min TurnoutTimeSeconds_min
cols_cible [['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max']]
cols_cible_minutes [['TurnoutTimeMinutes_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max']]
[    -60      60     120     180     240     300     360     420     480
     540     600     900 1200000]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30]
TurnoutTimeMinutes_mean TurnoutTimeSeconds_mean
cols_cible [['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeS

,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,PropertyType_6,PropertyType_7,PropertyType_8,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,NumPumpsAttending,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max,TurnoutTimeMinutes_min,TurnoutTimeMinutes_mean,TurnoutTimeMinutes_max
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2.00,0,0,0,0,1,0,0,0,1,0,0,1,240.00,390.00,540.00,253.00,253.00,253.00,89.00,89.00,89.00,5,5,5
1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1.00,0,0,0,1,0,0,0,0,1,0,0,1,420.00,420.00,420.00,151.00,151.00,151.00,157.00,157.00,157.00,3,3,3
2,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1.00,0,0,0,1,0,0,0,0,1,0,0,1,720.00,720.00,720.00,108.00,108.00,108.00,102.00,102.00,102.00,2,2,2


(537368, 45)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'TurnoutTimeMinutes_min', 'TurnoutTimeMinutes_mean', 'TurnoutTimeMinutes_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
1054159,12,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,1
1054160,12,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,1


(537368, 18)
--------------------------------------------------------------------------------
TurnoutTimeSeconds_min >> TurnoutTimeMinutes_min
Resampling
429894
TurnoutTimeMinutes_min
2     213412
1     194825
3      17708
4       2138
5        867
6        416
7        224
8        122
9         72
15        67
10        33
30        10
Name: count, dtype: int64
Smote
TurnoutTimeMinutes_min
1     213412
2     213412
3     213412
4     213412
5     213412
6     213412
7     213412
8     213412
9     213412
10    213412
15    213412
30    213412
Name: count, dtype: int64
TurnoutTimeMinutes_min
1     213412
2     213412
3     213412
4     213412
5     213412
6     213412
7     213412
8     213412
9     213412
10    213412
15    213412
30    213412
Name: count, dtype: int64
Durée de RandomUnderSampler : 346.51 secondes
2560944
-------------------------------------------
GridSearchCV pour DecisionTreeClassifier
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Meilleurs paramèt

...

Classe prédite,1,2,3,4,5,6,7,8,9,10,15,30
Classe réelle,,,,,,,,,,,,
1,22485,6248,2773,5156,4779,3090,1940,1138,578,198,596,76
2,17807,8171,7828,6136,5090,3148,1856,1296,602,262,646,82
3,710,529,1409,664,432,272,187,128,64,27,75,11
4,143,46,97,78,59,49,16,9,11,2,10,1
5,62,24,29,24,24,14,9,9,3,2,2,1
6,32,10,15,16,17,10,9,7,2,0,1,1
7,12,4,7,11,7,1,4,2,0,2,2,0
8,20,1,3,3,4,5,0,3,1,0,1,0
9,7,1,2,2,1,1,2,2,0,0,1,0


--------------------------------------------------------------------------------
TurnoutTimeSeconds_mean >> TurnoutTimeMinutes_mean
Resampling
429894
TurnoutTimeMinutes_mean
2     251829
1     150217
3      22851
4       2877
5       1083
6        479
7        238
8        130
9         77
15        67
10        36
30        10
Name: count, dtype: int64
Smote
TurnoutTimeMinutes_mean
1     251829
2     251829
3     251829
4     251829
5     251829
6     251829
7     251829
8     251829
9     251829
10    251829
15    251829
30    251829
Name: count, dtype: int64
TurnoutTimeMinutes_mean
1     251829
2     251829
3     251829
4     251829
5     251829
6     251829
7     251829
8     251829
9     251829
10    251829
15    251829
30    251829
Name: count, dtype: int64
Durée de RandomUnderSampler : 217.26 secondes
3021948
-------------------------------------------
GridSearchCV pour DecisionTreeClassifier
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Meilleurs paramètres {'cr

Classe prédite,1,2,3,4,5,6,7,8,9,10,15,30
Classe réelle,,,,,,,,,,,,
1,15497,5734,1777,4600,3882,2694,1491,861,543,185,445,65
2,19565,11089,7969,7867,6295,4148,2282,1480,805,344,757,90
3,801,609,1810,864,584,431,249,164,87,32,93,13
4,169,84,125,110,87,64,25,17,15,3,17,2
5,66,28,34,42,31,15,12,12,5,2,5,1
6,39,12,19,23,17,13,12,7,2,1,3,1
7,15,3,9,13,5,6,3,2,0,2,2,0
8,13,4,1,7,4,6,1,2,1,0,1,0
9,4,2,4,2,1,1,2,2,0,0,1,0


-------------------------------------------
GridSearchCV pour ExtraTreeClassifier
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Meilleurs paramètres {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 5}
Meilleur score 0.7060622158790922
Accuracy : 0.2657014719839217


KeyboardInterrupt: 

In [ ]:

Voting Classifier
Stacking Classifier

	# Define base classifiers
clf1 = LogisticRegression(random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = SVC(probability=True, random_state=42)
	# Define VotingClassifier
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3)], voting='soft')
	# Define parameter grid
param_grid = {
    'lr__C': [0.1, 1, 10],
    'dt__max_depth': [3, 5, 7],
    'svc__C': [0.1, 1, 10],
    'voting': ['hard', 'soft']
}
	# Perform grid search
grid_search = GridSearchCV(estimator=voting_clf,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
	# Report best score and parameters
print(f"Best score: {grid_search.best_score_:.3f}")
print(f"Best parameters: {grid_search.best_params_}")
	# Evaluate on test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print(f"Test set accuracy: {accuracy:.3f}")


XGBoost 
LCEClassifier

from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

rnd_clf = RandomForestClassifier(n_estimators=100,random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100,random_state=42)
svm_clf = SVC(max_iter=100,tol=20,random_state=42)
mlp_clf = MLPClassifier(random_state=42)

estimators = [rnd_clf,extra_trees_clf,svm_clf,mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train,y_train)
    
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("random_forest_clf",rnd_clf),
    ("Extra_trees_clf",extra_trees_clf),
    ("SVM",svm_clf),
    ("MLP",mlp_clf)
]

voting_clf = VotingClassifier(named_estimators)

print(voting_clf.fit(X_train,y_train))
print("voting_clf.score:",voting_clf.score(X_test,y_test))